In [1]:
import os

from forte.data.data_pack import DataPack
from forte.pipeline import Pipeline
from forte.data.data_pack import DataPack
from forte.pipeline import Pipeline
from typing import Any, Iterator

from ft.onto.payload_ontology import ImagePayload

from PIL import Image
from forte.data.base_reader import PackReader

/home/murphy/anaconda3/envs/forte_org_env/lib/python3.7/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/media/NAS/NLP/Petuum/forte/forte/data/ontology/ontology_code_generator.py:406: UserWarning: No `__init__` function found in the class Meta of the module <module 'forte.data.ontology.top' from '/media/NAS/NLP/Petuum/forte/forte/data/ontology/top.py'>.
  f"No `__init__` function found in the class"


In [2]:
class OcrReader(PackReader):
    r"""
    :class:`StringReader` is designed to read in a list of string variables.
    """

    # pylint: disable=unused-argument
    def _cache_key_function(self, collection) -> str:
        return str(hash(collection)) + ".html"

    def _collect(self, image_path) -> Iterator[Any]:  # type: ignore
        r"""Should be called with param ``Image_directory`` which is a path to a
        folder containing Image files.

        Args:
            Image_directory: Image directory containing the files.

        Returns: Iterator over paths to Image files
        """
        # construct ImageMeta and store it in DataPack
        return image_path

    def _parse_pack(self, data_source: str) -> Iterator[DataPack]:
        r"""Takes a raw string and converts into a :class:`~forte.data.data_pack.DataPack`.

        Args:
            data_source: str that contains text of a document.

        Returns: :class:`~forte.data.data_pack.DataPack` containing Document.
        """
        pack: DataPack = DataPack()
        payload_idx = 0
        # Read in Image data and store in DataPack
        # add Image payload into DataPack.payloads
        ip = ImagePayload(pack, payload_idx)
        try:
            import pytesseract
        except ModuleNotFoundError as e:
            raise ModuleNotFoundError(
                "OcrReader requires 'pytesseract' package to be installed."
            ) from e
        image_data = Image.open(data_source) # lazy loading
        ip.set_cache(image_data)
        ocr_text = pytesseract.image_to_string(image_data)
        pack.set_text(ocr_text)
        pack.pack_name = data_source
        
        yield pack


# TODO: split ocr part into a processor

In [5]:
pipeline = Pipeline[DataPack]()
pipeline.set_reader(
    OcrReader()
)
pipeline.initialize()

for pack in pipeline.process_one(["ocr.jpg"]):
    print(pack.text)



/media/NAS/NLP/Petuum/forte/forte/data/ontology/ontology_code_generator.py:406: UserWarning: No `__init__` function found in the class Meta of the module <module 'forte.data.ontology.top' from '/media/NAS/NLP/Petuum/forte/forte/data/ontology/top.py'>.
  f"No `__init__` function found in the class"


> /tmp/ipykernel_51872/2424896200.py(47)_parse_pack()
     45         pack.pack_name = data_source
     46         import pdb; pdb.set_trace()
---> 47         print('dafdsad')
     48 
     49         yield pack

<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=810x539 at 0x7FB9489A2B90>
<PIL.JpegImagePlugin.JpegImageFile image mode=RGB size=810x539 at 0x7FB9489A2B90>


BdbQuit: 